In [23]:
%store -r s3_bucket
%store -r prefix
%store -r training_job_name

In [25]:
bo = 's3://{}/{}/batch-prediction'.format(
    s3_bucket, 
    prefix
)
batch_output = bo

In [26]:
from sagemaker.estimator import Estimator

estimator = Estimator.attach(training_job_name)


2021-06-01 10:45:57 Starting - Preparing the instances for training
2021-06-01 10:45:57 Downloading - Downloading input data
2021-06-01 10:45:57 Training - Training image download completed. Training in progress.
2021-06-01 10:45:57 Uploading - Uploading generated training model
2021-06-01 10:45:57 Completed - Training job completed


In [27]:
transformer = estimator.transformer(
    instance_count=1, 
    instance_type='ml.m4.xlarge', 
    output_path=batch_output
)

In [28]:
s3_test_data = 's3://{}/{}/{}'.format(
    s3_bucket, 
    prefix, 
    "test"
)

In [29]:
%%time

transformer.transform(
    data=s3_test_data, 
    data_type='S3Prefix',
    content_type='application/x-image'
)

transformer.wait()

.....................................Docker entrypoint called with argument(s): serve
[06/01/2021 11:03:06 INFO 139695239014208] loaded entry point class algorithm.serve.server_config:config_api
[06/01/2021 11:03:06 INFO 139695239014208] loading entry points
[06/01/2021 11:03:06 INFO 139695239014208] loaded request iterator application/x-image
[06/01/2021 11:03:06 INFO 139695239014208] loaded request iterator image/jpeg
[06/01/2021 11:03:06 INFO 139695239014208] loaded request iterator image/png
[06/01/2021 11:03:06 INFO 139695239014208] loaded response encoder application/json
[06/01/2021 11:03:06 INFO 139695239014208] loaded response encoder application/jsonlines
[06/01/2021 11:03:06 INFO 139695239014208] loaded entry point class algorithm:model
[06/01/2021 11:03:06 INFO 139695239014208] Number of server workers: 3
[2021-06-01 11:03:06 +0000] [1] [INFO] Starting gunicorn 19.7.1
[2021-06-01 11:03:06 +0000] [1] [INFO] Listening at: http://0.0.0.0:8080 (1)
[2021-06-01 11:03:06 +0000] [1

UnexpectedStatusException: Error for Transform job image-classification-2021-06-01-10-57-02-937: Failed. Reason: ClientError: See job logs for more information

In [30]:
!mkdir -p batch_transform_output
!aws s3 cp {transformer.output_path}/ batch_transform_output/ --recursive

download: s3://sagemaker-cookbook-bucket/image-experiments/batch-prediction/ACOUTVSBRZFQ.png.out to batch_transform_output/ACOUTVSBRZFQ.png.out
download: s3://sagemaker-cookbook-bucket/image-experiments/batch-prediction/ADYSTHMLQUFJ.png.out to batch_transform_output/ADYSTHMLQUFJ.png.out
download: s3://sagemaker-cookbook-bucket/image-experiments/batch-prediction/AGXQTWUVSZRM.png.out to batch_transform_output/AGXQTWUVSZRM.png.out
download: s3://sagemaker-cookbook-bucket/image-experiments/batch-prediction/ABNIQFCOZDYW.png.out to batch_transform_output/ABNIQFCOZDYW.png.out
download: s3://sagemaker-cookbook-bucket/image-experiments/batch-prediction/ACSINQKJVRLH.png.out to batch_transform_output/ACSINQKJVRLH.png.out
download: s3://sagemaker-cookbook-bucket/image-experiments/batch-prediction/AGOHRLKTMWEZ.png.out to batch_transform_output/AGOHRLKTMWEZ.png.out
download: s3://sagemaker-cookbook-bucket/image-experiments/batch-prediction/AEMFUYPHGNSX.png.out to batch_transform_output/AEMFUYPHGNSX.

In [31]:
def get_class_from_batch_results(results_prob_list):
    best_index = results_prob_list.index(
        max(results_prob_list)
    )
    
    return {
        0: "ZERO",
        1: "ONE",
        2: "TWO",
        3: "THREE",
        4: "FOUR",
        5: "FIVE",
        6: "SIX",
        7: "SEVEN",
        8: "EIGHT",
        9: "NINE"
    }[best_index]

In [32]:
import json
filenames = !ls -1 batch_transform_output

for filename in filenames:
    c = ! cat batch_transform_output/{filename}
    content = json.loads(c[0])
    results_prob_list = content["prediction"]
    print(get_class_from_batch_results(
        results_prob_list
    ))

ZERO
TWO
TWO
SEVEN
SIX
SEVEN
SIX
TWO
ONE
SEVEN
ONE
ZERO
NINE
TWO
ONE
TWO
NINE
SEVEN
FOUR
TWO
NINE
SEVEN
ONE
FOUR
ONE
NINE
THREE
THREE
THREE
SEVEN
THREE
SEVEN
SEVEN
TWO
ONE
TWO
ONE
THREE
THREE
TWO
THREE
TWO
NINE
FOUR
SEVEN
THREE
NINE
ONE
TWO
SEVEN
THREE
FOUR
FOUR
SIX
EIGHT
FOUR
ZERO
FOUR
SEVEN
NINE
SIX
NINE
ONE
TWO
EIGHT
SEVEN
FOUR
NINE
ZERO
FIVE
FOUR
ONE
SIX
NINE
TWO
ONE
THREE
FIVE
ZERO
TWO
FOUR
FOUR
SEVEN
ONE
FOUR
THREE
FIVE
FOUR
ONE
ZERO
EIGHT
FIVE
FOUR
TWO
TWO
THREE
THREE
ONE
SEVEN
FIVE
ONE
TWO
FOUR
ONE
THREE
TWO
ONE
SIX
ONE
ONE
SIX
FIVE
ZERO
THREE
ONE
SEVEN
FOUR
ONE
TWO
ONE
SEVEN
NINE
ONE
ONE
SEVEN
TWO
FOUR
FOUR
THREE
ZERO
NINE
TWO
THREE
TWO
ONE
THREE
TWO
ONE
ONE
THREE
EIGHT
ZERO
TWO
ZERO
THREE
THREE
FOUR
THREE
SIX
NINE
ONE
NINE
EIGHT
FOUR
ZERO
ONE
ONE
ZERO
SIX
THREE
FIVE
ONE
NINE
ONE
ONE
SEVEN
ONE
SEVEN
TWO
FOUR
SEVEN
THREE
TWO
ZERO
FOUR
THREE
SEVEN
SEVEN
NINE
THREE
FOUR
NINE
SEVEN
SEVEN
SEVEN
FIVE
TWO
EIGHT
ONE
NINE
FOUR
FOUR
FIVE
TWO
FOUR
FOUR
NINE
THREE
THREE
TWO
SEVEN
ZERO
TWO

KeyError: 'prediction'